In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import argparse
import sys,os
import tensorflow as tf
from PIL import Image
from tqdm import tqdm

In [2]:
t=tqdm(pd.read_csv('test.csv').values)
test=[]
i=0
for tt in t:
    test.append(tt[0])
    i+=1

100%|██████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<?, ?it/s]


In [3]:
test

['ProstateX-0168-Lesion1-T2Axial-Class1',
 'ProstateX-0172-Lesion4-T2Axial-Class1',
 'ProstateX-0179-Lesion1-T2Axial-Class2',
 'ProstateX-0179-Lesion2-T2Axial-Class2',
 'ProstateX-0181-Lesion1-T2Axial-Class5',
 'ProstateX-0182-Lesion1-T2Axial-Class1',
 'ProstateX-0183-Lesion1-T2Axial-Class3',
 'ProstateX-0184-Lesion2-T2Axial-Class3',
 'ProstateX-0186-Lesion1-T2Axial-Class2',
 'ProstateX-0187-Lesion1-T2Axial-Class3',
 'ProstateX-0188-Lesion1-T2Axial-Class1',
 'ProstateX-0190-Lesion1-T2Axial-Class2',
 'ProstateX-0191-Lesion1-T2Axial-Class1',
 'ProstateX-0192-Lesion1-T2Axial-Class4',
 'ProstateX-0193-Lesion4-T2Axial-Class2',
 'ProstateX-0195-Lesion1-T2Axial-Class4',
 'ProstateX-0196-Lesion1-T2Axial-Class5',
 'ProstateX-0196-Lesion2-T2Axial-Class5',
 'ProstateX-0196-Lesion3-T2Axial-Class3',
 'ProstateX-0198-Lesion1-T2Axial-Class5',
 'ProstateX-0199-Lesion1-T2Axial-Class4',
 'ProstateX-0199-Lesion2-T2Axial-Class4',
 'ProstateX-0200-Lesion1-T2Axial-Class5',
 'ProstateX-0201-Lesion1-T2Axial-C

In [4]:
def load_image(filename):
    #Read in the image_data to be classified."""
    return tf.gfile.FastGFile(filename, 'rb').read()

def load_labels(filename):
    #Read in labels, one label per line."""
    return [line.rstrip() for line in tf.gfile.GFile(filename)]

def load_graph(filename):
    #Unpersists graph from file as default graph."""
    with tf.gfile.FastGFile(filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

In [5]:
def run_graph(src, dest, labels, input_layer_name, output_layer_name, num_top_predictions):
    with tf.Session() as sess:
        # Feed the image_data as input to the graph.
        # predictions  will contain a two-dimensional array, where one
        # dimension represents the input image count, and the other has
        # predictions per class
        i=0
        #with open('submit.csv','w') as outfile:
        for f in test:
            im=Image.open(os.path.join(src,f+'.png'))
            img=im.convert('RGB')
            img.save(os.path.join(dest,test[i]+'.jpg'))
            image_data=load_image(os.path.join(dest,test[i]+'.jpg'))
            softmax_tensor=sess.graph.get_tensor_by_name(output_layer_name)
            predictions,=sess.run(softmax_tensor, {input_layer_name: image_data})
            # Sort to show labels in order of confidence             
            top_k = predictions.argsort()[-num_top_predictions:][::-1]
            for node_id in top_k:
                predicted_label = labels[node_id]
                score = predictions[node_id]
                print(test[i]+',',predicted_label+','+str(score))
                #outfile.write(test[i]+','+human_string+'\n')
            i+=1

In [6]:
src=os.path.join('.','test_img')
dest=os.path.join('.','test_img2')
labels='D:/tmp/prostate-experiment-b/output_labels_prostate-baseL.txt'
graph='D:/tmp/prostate-experiment-b/output_graph_prostate-baseL.pb'
input_layer='DecodeJpeg/contents:0'
output_layer='final_result:0'
num_top_predictions=1
labels = load_labels(labels)
load_graph(graph)
run_graph(src,dest,labels,input_layer,output_layer,num_top_predictions)

Instructions for updating:
Use tf.gfile.GFile.
ProstateX-0168-Lesion1-T2Axial-Class1, cancer,0.769856
ProstateX-0172-Lesion4-T2Axial-Class1, cancer,0.5244625
ProstateX-0179-Lesion1-T2Axial-Class2, cancer,0.90302575
ProstateX-0179-Lesion2-T2Axial-Class2, cancer,0.8920897
ProstateX-0181-Lesion1-T2Axial-Class5, cancer,0.9599832
ProstateX-0182-Lesion1-T2Axial-Class1, cancer,0.84712803
ProstateX-0183-Lesion1-T2Axial-Class3, cancer,0.74540156
ProstateX-0184-Lesion2-T2Axial-Class3, healthy,0.59150594
ProstateX-0186-Lesion1-T2Axial-Class2, cancer,0.83064085
ProstateX-0187-Lesion1-T2Axial-Class3, cancer,0.95105535
ProstateX-0188-Lesion1-T2Axial-Class1, healthy,0.63189936
ProstateX-0190-Lesion1-T2Axial-Class2, cancer,0.60650474
ProstateX-0191-Lesion1-T2Axial-Class1, cancer,0.64350057
ProstateX-0192-Lesion1-T2Axial-Class4, cancer,0.6918193
ProstateX-0193-Lesion4-T2Axial-Class2, cancer,0.6506637
ProstateX-0195-Lesion1-T2Axial-Class4, cancer,0.9734996
ProstateX-0196-Lesion1-T2Axial-Class5, healthy,